# Recency Frequency Monetary (RFM) Analysis

In [ ]:
import math
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from scipy.stats import norm
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import missingno as msno

import warnings
warnings.filterwarnings('ignore')

### Dataset

This is a transactional data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.
	
https://archive.ics.uci.edu/ml/datasets/online+retail

**Attribute Information**:

    InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
    StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
    Description: Product (item) name. Nominal.
    Quantity: The quantities of each product (item) per transaction. Numeric.
    InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
    UnitPrice: Unit price. Numeric, Product price per unit in sterling.
    CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
    Country: Country name. Nominal, the name of the country where each customer resides. 

In [ ]:
#df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df = pd.read_excel('Online Retail.xlsx')
df.head()

In [ ]:
df.dtypes

## Data Cleanup

In [ ]:
df.describe()

In [ ]:
#plot missing data
msno.matrix(df)
plt.show(plt.figure(figsize=(5, 3)))

In [ ]:
# get the missing data

In [ ]:
# drop Description column

In [ ]:
# drop rows with missing InvoiceNo
# drop rows with missing InvoiceDate
# drop rows with missing CustomerId
df.shape[0]

In [ ]:
#fix datatypes
df['CustomerID'] = df['CustomerID'].astype(str)
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
# Convert 'InvoiceDate' column to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
#remove the credit transactions (those with invoice numbers containing C).
df = df[~df['InvoiceNo'].str.contains('C')]
df.shape[0]

In [ ]:
df = df[df['UnitPrice'] > 0]
df.shape[0]

In [ ]:
df.describe()

## Exploratory Data Analysis

In [ ]:
##countries

In [ ]:
#Number of customers

In [ ]:
#Number of stockcodes

In [ ]:
#transcations by country

In [ ]:
# TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.sort_values(by='TotalPrice', ascending=False).head()

In [ ]:
# Visualize the distribution of TotalPrice

## RFM Analysis
Calculation of RFM metrics (recency, frequency, and monetary)
    
    recency = date of analysis - purchase date of the relevant customer
    frequency = customer's total number of purchases
    monetary = total monetary value as a result of the customer's total purchases
    
We will assume the date of analysis was 1 days after the max(date) in the dataset

In [ ]:
# get the last invoice date in the dataset
max_date = df['InvoiceDate'].max()
max_date

In [ ]:
# adding a day to the calculated max date as date of analysis
snapshot_date = max_date + dt.timedelta(days = 1)
snapshot_date

In [ ]:
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, # Recency
    'InvoiceNo': 'count',                                    # Frequency
    'TotalPrice': 'sum' ,                                    # Monetary
})

rfm.rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalPrice': 'Monetary'}, inplace=True)

# Display the RFM table
rfm

### Segmenting customers based on RFM scores

Create R, F, M scores from 1 (least) to 4 (higest)

<code>r_quartiles:</code> Generate labels in reverse order from 4 to 1. This means that the quartiles will be labeled as 4 for the first quartile (highest recency), 3 for the second quartile, 2 for the third quartile, and 1 for the fourth quartile (lowest recency). This reverse ordering might be chosen for interpretability, where higher values indicate better or more recent behavior.

In [ ]:
# Define quartiles for Recency, Frequency, and Monetary
r_quartiles = pd.qcut(rfm['Recency'], q=4, labels=range(4, 0, -1))
f_quartiles = pd.qcut(rfm['Frequency'], q=4, labels=range(1, 5))
m_quartiles = pd.qcut(rfm['Monetary'], q=4, labels=range(1, 5))

rfm = rfm.assign(R=r_quartiles, F=f_quartiles, M=m_quartiles)

# Calculate RFM score
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)
#rfm["RFM_Score"] = 0.6*rfm['F'] + 0.2*rfm['F'] + 0.2*rfm['M'] #alternative weighted score

# Define RFM segments
rfm['RFM_Segment'] = rfm[['R', 'F', 'M']].apply(lambda x: ''.join(map(str, x)), axis=1)

# Display the RFM table with segments
rfm

In [ ]:
#recency
rfm.Recency.describe()

In [ ]:
#Recency plot
x = rfm.Recency
mu = np.mean(rfm.Recency)
sigma = math.sqrt(np.var(rfm.Recency))
n, bins, patches = plt.hist(x, 400, facecolor='green', alpha=0.75)
plt.xlabel('Recency in days')
plt.ylabel('Number of transactions')
plt.title(r'Histogram of sales recency')
plt.grid(True)

In [ ]:
rfm.groupby("RFM_Segment").count()

In [ ]:
#humanization
# rfm.loc[(rfm['RFM_Score'] <= 4), 'Segment'] = 'Silver'
# rfm.loc[(rfm['RFM_Score'] > 4) & (rfm['RFM_Score'] <= 7), 'Segment'] = 'Gold'
# rfm.loc[(rfm['RFM_Score'] > 7) & (rfm['RFM_Score'] <= 10), 'Segment'] = 'Platinum'
# rfm.loc[(rfm['RFM_Score'] > 10), 'Segment'] = 'Diamond'

rfm.head()

In [ ]:
# Calculate correlation matrix
rfm_corr = rfm[['R' ,'F' ,'M']].corr()

# Create a heatmap
plt.figure(figsize=(5, 3))
sns.heatmap(rfm_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
sns.pairplot(rfm[['Recency' ,'Frequency' ,'Monetary','Segment']], hue="Segment")
plt.show()

In [ ]:
#group by topic and plot rows in the descending order
rfm.groupby('Segment').size().sort_values(ascending=True)\
    .plot(title="Number of Customers", kind='barh',figsize=(5,2))
plt.show()

## Sample Marketing Strategy

**Gold Customers:**
    
    - Exclusive Access: Early access to limited-edition products or special collections before they are available to other customers.
    - VIP Discounts: Higher discounts or special promotional codes applicable to select products or categories.
    - Dedicated Support: Assign a dedicated customer support representative or hotline to address their inquiries and resolve issues promptly.

**Silver Customers:**
    
    - Tier Upgrade Incentives: Incentives such as a one-time discount or bonus points for reaching the Gold tier.
    - Tier-specific Offers: Targeted offers on products complementary to their past purchases or interests.
    - Early Sale Access: Early access to seasonal sales or clearance events to encourage repeat purchases.

**Platinum Customers:**
    
    - Personal Shopper Service: Introduce a personalized shopping service with recommendations and assistance from a dedicated personal shopper.
    - Exclusive Events: Exclusive VIP events, such as product launches or fashion shows, with complimentary refreshments and VIP seating.
    - Customized Products: Option to customize products with monograms, engravings, or bespoke designs tailored to their preferences.

**Diamond Customers:**
    
    - Luxury Gifts: Surprise with luxury gifts or limited-edition items as a token of appreciation for their loyalty.
    - Private Consultations: Arrange private consultations with designers or industry experts to discuss bespoke or high-end purchases.
    - Elite Benefits: Elite benefits such as complimentary expedited shipping, priority access to customer support,

## Cluster Analysis 

Behavioral clustering involves grouping customers based on similarities in their behavior, preferences, or characteristics. It helps identify distinct customer segments that may not be evident from RFM analysis alone. 

Combining clustering and RFM analysis provides complementary insights into customer behavior. Clustering helps identify distinct customer segments, while RFM analysis identifies high-value customers within those segments. After clustering customers into segments, RFM analysis can be applied within each segment to identify differences in recency, frequency, and monetary value. This allows for segment-specific marketing strategies tailored to the unique characteristics of each cluster.

In [ ]:
# Cluster the data based on RFM categories. Could we use a more elaborate clustering scheme ?


In [ ]:
#k=4 gets better clusters, however take inputs from the business


In [ ]:
#group by topic and plot rows in the descending order
rfm.groupby('Cluster').size().sort_values(ascending=True)\
    .plot(title="Number of Customers", kind='barh',figsize=(5,2))
plt.show()